# Applied Data Science Capstone

## This Notebook will be used for the Coursera capstone project

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project, we will try to find an optimal location for a business operation. Specifically, a close friend of mine asked me to help him with finding a good place for opening a **cafe in Athina, Greece**. I live in Athina, so it's easy for me to help him with this issue.

It doesn't matter in which borough in Athina his cafe is going to open as soon as it is in the center of a borough with a lot of similar shops.

We will use our data science powers to cluster boroughs in Athina based on the abovementioned criteria. I hope that my work will help him to choose a pretty good location for his cafe.

## Data <a name="data"></a>

Based on the definition of our problem, the factors that will influence our decision are:
* Number of the existing cafe in the boroughs (we need cafe shop to be the first or second popular shop in a borough center)
* Similar boroughs in Athina (so my friend can choose where he prefers to open his cafe) 

I decided to use the coordinates of Athina boroughs centers and search around them (range < 500m) for shops.

I used data sources as follows:
* Borough centers obtained from 
 http://geodata.gov.gr/maps/?package=7a9cb4c4-f192-4df8-a6b4-1143c80ebc2d&resource=525ed8ae-1f2c-4c14-acb5-14d2d00be4a4&locale=el , which is a formally Greece government site
* number of shops and their type and location in every borough center obtained using **Foursquare API**

### Import Libraries

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Load excel file and create the dataframe

##### I download this file from http://geodata.gov.gr/maps/?package=7a9cb4c4-f192-4df8-a6b4-1143c80ebc2d&resource=525ed8ae-1f2c-4c14-acb5-14d2d00be4a4&locale=el

In [3]:
df = pd.read_excel("athens coordinates.xlsx")
df.head()

,Neighborhood,Latitude,Longitude
0,d. agias paraskeuhs kentro,38.0132,23.8210
1,d. agias paraskeuhs pl. ag. ioannh,38.0055,23.8280
2,d. agias paraskeuhs boreia,38.0183,23.8288
3,d. amarousiou kentro,38.0562,23.8068
4,d. amarousiou oaka,38.0373,23.7946


##### Check dataframe shape

In [4]:
df.shape

(104, 3)

So now we have all 104 boroughs centers in Athina city and we proceed in the analysis.

## Methodology <a name="methodology"></a>

In this project, we will direct our efforts on detecting borough centers that have high coffee shop density ~500m around the borough center.

In the first step, we have collected the required data, borough centers coordinates.

Accordingly, we collect information about shops around those centers, according to Foursquare categorization.

Then, we cluster boroughs into five clusters.

Next step, we choose the cluster that has a cafe shop as a first or second popular shop.

Finally, we propose one of these boroughs as an optimal place to open a cafe.

## Analysis <a name="analysis"></a>

### Generate maps to visualize neighborhoods of Athina Greece

#### Use geopy library to get the latitude and longitude values of Athina

In [13]:
address = 'Athina City, ATH'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Athina City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Athina City are 38.00423275, 23.72378525842142.


#### Create a map of Athina with neighborhoods superimposed on top

In [16]:
# create map of Athina using latitude and longitude values
map_athina = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_athina)  
    
map_athina

#### Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'PKKZ00I0FIFWGJYIT3VFOO3P323YVTK2C4USL2SOZWNXGWXD' # your Foursquare ID
CLIENT_SECRET = '1YNIZ2QUSFHUQZGF5K0CWP5ZKLADZSXV1BVFHDQP0RMZ1MZY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PKKZ00I0FIFWGJYIT3VFOO3P323YVTK2C4USL2SOZWNXGWXD
CLIENT_SECRET:1YNIZ2QUSFHUQZGF5K0CWP5ZKLADZSXV1BVFHDQP0RMZ1MZY


### Explore Neighborhoods in Athina

#### Create a function to explore all the neighborhoods in Athina

In [20]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Write the code to run the above function on each neighborhood and create a new dataframe called *athina_venues*.

In [21]:
athina_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])

d. agias paraskeuhs kentro
d. agias paraskeuhs pl. ag. ioannh
d. agias paraskeuhs boreia
d. amarousiou kentro
d. amarousiou oaka
d. brilhssion
melissia
d. hrakleiou kentro
d. hrakleiou boreia
d. khfisias kentro
d. khfisias bd
nea eruthraia
d. lukobrushs
d. peukhs
d. metamorfoseos
d. neas ionias kentro
d. neas ionias perissos
d. neas filadelfeias kentro
d. neas filadelfeias boreio
d.papagou
d. xolargou
d. pentelhs
d. neas pentelhs
d. filothehs
d. neou cuxikou
d. cuxikou
d. xalandriou poludroso
d. xalandriou kentro
d. xalandriou kato
d. agias barbaras
d. agion anarguron - kamaterou
nea xalkhdona
d. aigaleo
d. iliou
d. peristeriou anthoupolh
d. peristeriou kentro
d. peristeriou ag. antonios
d. peristeriou mpournazi
d. petroupolhs
d. xaϊdariou karaϊskakh
d. xaϊdariou dasos
kamatero
liosia
axarnes
axarnes notia
d. athhnaion an. pathsia
d. athhnaion kato pathsia
d. athhnaion kucelh
d. athhnaion pl attikhs
d. athhnaion sepolia
d. athhnaion biktoria
d. athhnaion metaξourgeio
d. athhnaion keram

#### Check the size of the resulting dataframe

In [22]:
athina_venues.shape

(4282, 7)

In [23]:
athina_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,d. agias paraskeuhs kentro,38.0132,23.821,Daddy's bread,38.012745,23.820685,Bakery
1,d. agias paraskeuhs kentro,38.0132,23.821,WallStreet Food and Bar,38.014651,23.821249,Restaurant
2,d. agias paraskeuhs kentro,38.0132,23.821,pet4u,38.013986,23.819567,Pet Store
3,d. agias paraskeuhs kentro,38.0132,23.821,Starbucks,38.013070,23.819316,Coffee Shop
4,d. agias paraskeuhs kentro,38.0132,23.821,Creperie Asterix,38.013697,23.822646,Creperie


#### Analyze Each Neighborhood

In [24]:
# one hot encoding
athina_onehot = pd.get_dummies(athina_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
athina_onehot['Neighborhood'] = athina_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot[['Neighborhood']]] + list(toronto_onehot.columns)
#toronto_onehot = toronto_onehot[fixed_columns]

athina_onehot.head()

,ATM,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bistro,Board Shop,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Kitchen Supply Store,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Nursery School,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Ouzeri,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Patsa Restaurant,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Professional & Other Places,Pub,Public Art,Record Shop,Recording Studio,Recreation Center,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tsipouro Restaurant,Turkish Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
athina_grouped = athina_onehot.groupby('Neighborhood').mean().reset_index()
athina_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bistro,Board Shop,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Kitchen Supply Store,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Nursery School,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Ouzeri,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Patsa Restaurant,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Professional & Other Places,Pub,Public Art,Record Shop,Recording Studio,Recreation Center,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tsipouro Restaurant,Turkish Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,axarnes,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

#### Write a function to sort the venues in descending order

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = athina_grouped['Neighborhood']

for ind in np.arange(athina_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(athina_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,axarnes,Café,Mobile Phone Shop,Gym,Plaza,Pub,Meze Restaurant,Cocktail Bar,Bakery,Coffee Shop,Supermarket
1,axarnes notia,Dessert Shop,Light Rail Station,Hotel,Basketball Court,Supermarket,Café,Food Court,Food Service,Food Stand,Food Truck
2,d. agias barbaras,Greek Restaurant,Café,Grocery Store,Jewelry Store,Park,Taverna,Optical Shop,Bakery,Fish Taverna,Coffee Shop
3,d. agias paraskeuhs boreia,Grocery Store,Pool Hall,Bakery,Plaza,Greek Restaurant,Gym / Fitness Center,Basketball Court,Bookstore,Hotel Pool,Gaming Cafe
4,d. agias paraskeuhs kentro,Coffee Shop,Electronics Store,Bakery,Café,Supermarket,Pizza Place,Clothing Store,Pharmacy,Baby Store,Cosmetics Shop


### Cluster Neighborhoods

#### Run *k*-means to cluster the neighborhood into 5 clusters

In [28]:
# set number of clusters
kclusters = 5

athina_grouped_clustering = athina_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(athina_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 3, 1, 1, 3, 2, 1, 4, 2])

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

athina_merged = df

# merge athina_grouped with athina_data to add latitude/longitude for each neighborhood
athina_merged = athina_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

athina_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,d. agias paraskeuhs kentro,38.0132,23.8210,1,Coffee Shop,Electronics Store,Bakery,Café,Supermarket,Pizza Place,Clothing Store,Pharmacy,Baby Store,Cosmetics Shop
1,d. agias paraskeuhs pl. ag. ioannh,38.0055,23.8280,3,Café,Greek Restaurant,Bar,Sushi Restaurant,Gym / Fitness Center,Tennis Stadium,Pub,Coffee Shop,Grilled Meat Restaurant,Nightclub
2,d. agias paraskeuhs boreia,38.0183,23.8288,1,Grocery Store,Pool Hall,Bakery,Plaza,Greek Restaurant,Gym / Fitness Center,Basketball Court,Bookstore,Hotel Pool,Gaming Cafe
3,d. amarousiou kentro,38.0562,23.8068,2,Café,Bar,Bakery,Souvlaki Shop,Coffee Shop,Office,Music Venue,Multiplex,Cocktail Bar,Plaza
4,d. amarousiou oaka,38.0373,23.7946,1,Clothing Store,Café,Electronics Store,Cosmetics Shop,Basketball Court,Lingerie Store,Gym / Fitness Center,Coffee Shop,Supermarket,Sporting Goods Shop


#### Visualize the resulting clusters

In [30]:
#toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0.0).astype(int)

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(athina_merged['Latitude'], athina_merged['Longitude'], athina_merged['Neighborhood'], athina_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [39]:
athina_merged.loc[athina_merged['Cluster Labels'] == 0, athina_merged.columns[[0] + list(range(3, athina_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,d. pentelhs,0,Bus Stop,Bakery,Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Yoga Studio


#### Cluster 2

In [40]:
athina_merged.loc[athina_merged['Cluster Labels'] == 1, athina_merged.columns[[0] + list(range(3, athina_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,d. agias paraskeuhs kentro,1,Coffee Shop,Electronics Store,Bakery,Café,Supermarket,Pizza Place,Clothing Store,Pharmacy,Baby Store,Cosmetics Shop
2,d. agias paraskeuhs boreia,1,Grocery Store,Pool Hall,Bakery,Plaza,Greek Restaurant,Gym / Fitness Center,Basketball Court,Bookstore,Hotel Pool,Gaming Cafe
4,d. amarousiou oaka,1,Clothing Store,Café,Electronics Store,Cosmetics Shop,Basketball Court,Lingerie Store,Gym / Fitness Center,Coffee Shop,Supermarket,Sporting Goods Shop
5,d. brilhssion,1,Park,Convenience Store,Betting Shop,Basketball Court,Playground,Performing Arts Venue,Soccer Stadium,Grocery Store,Bakery,Event Space
10,d. khfisias bd,1,Bistro,Video Store,Fish Taverna,Auto Workshop,Coffee Shop,Turkish Restaurant,Bar,Park,Steakhouse,Frozen Yogurt Shop
11,nea eruthraia,1,Dessert Shop,Seafood Restaurant,Clothing Store,Sushi Restaurant,Bar,Gym / Fitness Center,Ice Cream Shop,Fish Taverna,Coffee Shop,Cocktail Bar
16,d. neas ionias perissos,1,Supermarket,Plaza,Pub,Souvlaki Shop,Snack Place,Bakery,Fast Food Restaurant,Paper / Office Supplies Store,Toy / Game Store,Dry Cleaner
18,d. neas filadelfeias boreio,1,Souvlaki Shop,Athletics & Sports,Smoke Shop,Food Truck,Bus Stop,Gourmet Shop,Seafood Restaurant,Tennis Court,Chinese Restaurant,Toy / Game Store
19,d.papagou,1,Park,Bakery,Playground,Dessert Shop,Tennis Court,Souvlaki Shop,Café,Camera Store,Restaurant,Pharmacy
20,d. xolargou,1,Cocktail Bar,Dessert Shop,Electronics Store,Greek Restaurant,Clothing Store,Café,Burger Joint,Grilled Meat Restaurant,Mexican Restaurant,Meze Restaurant


#### Cluster 3

In [41]:
athina_merged.loc[athina_merged['Cluster Labels'] == 2, athina_merged.columns[[0] + list(range(3, athina_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,d. amarousiou kentro,2,Café,Bar,Bakery,Souvlaki Shop,Coffee Shop,Office,Music Venue,Multiplex,Cocktail Bar,Plaza
6,melissia,2,Bar,Café,Pizza Place,Coffee Shop,Taverna,Supermarket,Souvlaki Shop,Grilled Meat Restaurant,African Restaurant,Creperie
9,d. khfisias kentro,2,Café,Jewelry Store,Gourmet Shop,Cosmetics Shop,Coffee Shop,Bakery,Greek Restaurant,Lingerie Store,Italian Restaurant,Bookstore
13,d. peukhs,2,Café,Bakery,Mobile Phone Shop,Coffee Shop,Jewelry Store,Playground,Betting Shop,Souvlaki Shop,Bookstore,Snack Place
14,d. metamorfoseos,2,Souvlaki Shop,Café,Bookstore,Supermarket,Toll Booth,Motorcycle Shop,Auto Workshop,Donut Shop,Electronics Store,Ouzeri
17,d. neas filadelfeias kentro,2,Bar,Café,Greek Restaurant,Meze Restaurant,Souvlaki Shop,Creperie,Coffee Shop,Taverna,Snack Place,Seafood Restaurant
26,d. xalandriou poludroso,2,Bakery,Dessert Shop,Café,Gym,Taverna,Yoga Studio,Movie Theater,Greek Restaurant,Dance Studio,Video Store
27,d. xalandriou kentro,2,Bar,Café,Gym / Fitness Center,Dessert Shop,Greek Restaurant,Cocktail Bar,Coffee Shop,Pizza Place,Ice Cream Shop,Supermarket
30,d. agion anarguron - kamaterou,2,Plaza,Mobile Phone Shop,Café,Fast Food Restaurant,Coffee Shop,Bougatsa Shop,Snack Place,Supermarket,Bus Stop,Taverna
31,nea xalkhdona,2,Café,Sporting Goods Shop,Turkish Restaurant,Gym,Grilled Meat Restaurant,Food Court,Bakery,Sandwich Place,Meze Restaurant,Supermarket


#### Cluster 4

In [42]:
athina_merged.loc[athina_merged['Cluster Labels'] == 3, athina_merged.columns[[0] + list(range(3, athina_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,d. agias paraskeuhs pl. ag. ioannh,3,Café,Greek Restaurant,Bar,Sushi Restaurant,Gym / Fitness Center,Tennis Stadium,Pub,Coffee Shop,Grilled Meat Restaurant,Nightclub
7,d. hrakleiou kentro,3,Café,Greek Restaurant,Brewery,Snack Place,Meze Restaurant,Movie Theater,Coffee Shop,Plaza,Pizza Place,Pub
8,d. hrakleiou boreia,3,Café,Souvlaki Shop,Dance Studio,Dessert Shop,Gym,Supermarket,Video Store,Soccer Field,Burger Joint,Coffee Shop
12,d. lukobrushs,3,Greek Restaurant,Supermarket,Bakery,Coffee Shop,Café,Fruit & Vegetable Store,Gym,Pet Store,Ice Cream Shop,Nightclub
15,d. neas ionias kentro,3,Café,Greek Restaurant,Shoe Store,Gym,Pie Shop,Meze Restaurant,Bakery,Coffee Shop,Arts & Crafts Store,Gift Shop
29,d. agias barbaras,3,Greek Restaurant,Café,Grocery Store,Jewelry Store,Park,Taverna,Optical Shop,Bakery,Fish Taverna,Coffee Shop
33,d. iliou,3,Café,Greek Restaurant,Coffee Shop,Meze Restaurant,Bar,Burger Joint,Plaza,Bookstore,Gourmet Shop,Mobile Phone Shop
34,d. peristeriou anthoupolh,3,Café,Jewelry Store,Taverna,Gym,Grocery Store,Grilled Meat Restaurant,Park,Tailor Shop,Italian Restaurant,Volleyball Court
37,d. peristeriou mpournazi,3,Café,Greek Restaurant,Meze Restaurant,Nightclub,Gym,Bowling Alley,Supermarket,Ice Cream Shop,Creperie,Pool Hall
38,d. petroupolhs,3,Café,Bar,Supermarket,Nightclub,Bakery,Souvlaki Shop,Snack Place,Shoe Store,Seafood Restaurant,Meze Restaurant


#### Cluster 5

In [43]:
athina_merged.loc[athina_merged['Cluster Labels'] == 4, athina_merged.columns[[0] + list(range(3, athina_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,d. agiou ioannou renth,4,Soccer Field,Stadium,Tennis Court,Warehouse Store,Park,Athletics & Sports,Food Stand,Food,Food & Drink Shop,Food Court


## Results and Discussion <a name="results"></a>

Our analysis shows that cluster number 4 is the one with a high density of cafe shops, so these borough centers are candidates' places for our proposal.

We will take only those centers that have a cafe shop as the first common venue.

That choice narrows our results furthermore from 25 to 19 centers. 

Lastly, if we want something closer to Athina center, we have only 6 centers.

We can propose any one of those 6 for opening a cafe shop.

## Conclusion <a name="conclusion"></a>

The purpose of this project was to find for a friend of mine an optimal location for a business operation in Athina Greece, specifically a cafe shop. As my friend request, it doesn't matter in which borough in Athina his cafe is going to open as soon as it is in the center of a borough with a lot of similar shops. Our initial data includes 104 borough centers and after analysis, we cluster the centers into 5 clusters, we choose cluster number 4 as the most suitable and finally, we narrow down into 6 centers as optimal locations.
Of course, can be done more work and search these areas with more details.   